In [1]:
import re
import os, os.path
import fnmatch
import json
import collections
import glob
import json
# import modules for web scrapping
import requests
from bs4 import BeautifulSoup

In [2]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT

def get_keyword(docs):
    final_keywords = []
    vectorizer = KeyphraseCountVectorizer(pos_pattern= '<N.*>+')
    kw_model = KeyBERT()
    onewords = kw_model.extract_keywords(docs=docs, vectorizer=vectorizer,top_n=3,stop_words='english')
    return onewords

In [3]:
book ="SOMEWT"
dir_path = './%s/part'% book
rootdir = r"C:\Users\hp\OneDrive\Documents\topic_coherence\%s"% book
part = 1
mp = {}
for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
    x = 1
    no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
    while x <= no_of_chapters:
        target_x = no_of_chapters+1
        for i in range(x,target_x):
            with open(dir_path + str(part) + '/chapter'+ str(i) + '.txt', 'r', encoding='utf-8') as content_file:
                key = str(part) + '/chapter'+ str(i)                
                content = content_file.read()
                print(key)
                mp[key] = get_keyword(content)
        x = target_x
        part += 1

1/chapter1
1/chapter2
1/chapter3
1/chapter4
1/chapter5
1/chapter6
1/chapter7
1/chapter8
1/chapter9
1/chapter10
1/chapter11
1/chapter12
1/chapter13
1/chapter14
1/chapter15
1/chapter16
1/chapter17
1/chapter18
1/chapter19
1/chapter20
1/chapter21
1/chapter22
1/chapter23
1/chapter24
2/chapter1
2/chapter2
2/chapter3
2/chapter4
2/chapter5
2/chapter6
2/chapter7
2/chapter8
2/chapter9
2/chapter10
2/chapter11
2/chapter12
2/chapter13
2/chapter14
2/chapter15
2/chapter16
2/chapter17
2/chapter18
2/chapter19
2/chapter20
2/chapter21
2/chapter22
2/chapter23
2/chapter24
2/chapter25
2/chapter26
2/chapter27
2/chapter28
2/chapter29
3/chapter1
3/chapter2
3/chapter3
3/chapter4
3/chapter5
3/chapter6
3/chapter7
3/chapter8
3/chapter9
3/chapter10
3/chapter11
3/chapter12
3/chapter13
3/chapter14
3/chapter15
3/chapter16
3/chapter17
3/chapter18
3/chapter19
3/chapter20
3/chapter21
3/chapter22
3/chapter23
4/chapter1
4/chapter2
4/chapter3
4/chapter4
4/chapter5
4/chapter6
4/chapter7
4/chapter8
4/chapter9
4/chapter10
4/ch

In [4]:
mp2 = collections.defaultdict(list)
for key,val in mp.items():
    for word,score in val:
        if score>0.45:
            mp2[key].append(word)
print(mp2)

defaultdict(<class 'list'>, {'1/chapter1': ['karamchand gandhi', 'alias kaba gandhi', 'uttamchand gandhi'], '1/chapter4': ['faithfulness'], '1/chapter7': ['smoking stumps', 'cigarettes', 'cigarette'], '1/chapter9': ['vaishnava faith'], '1/chapter10': ['ahmedabad', 'matriculation examination', 'kathiawad students'], '1/chapter13': ['ringworm'], '1/chapter14': ['vegetarianism', 'dietary', 'diet'], '1/chapter16': ['vegetarianism', 'vegetarian'], '1/chapter21': ['narayan hemchandra', 'time narayan hemchandra', 'narayan'], '1/chapter22': ['eiffel tower'], '1/chapter24': ['legal maxims'], '2/chapter3': ['briefless barristers', 'barrister', 'barristers'], '2/chapter4': ['bombay commissions', 'barrister'], '2/chapter7': ['port natal'], '2/chapter8': ['christian indians'], '2/chapter11': ['prayers', 'prayer'], '2/chapter13': ['transvaal'], '2/chapter14': ['barrister', 'litigation'], '2/chapter15': ['prayer'], '2/chapter16': ['natal legislative assembly', 'durban'], '2/chapter17': ['sheth haji m

In [5]:
keyword_to_chap = collections.defaultdict(list)
for key,list_val in mp2.items():
    for val in list_val:
        keyword_to_chap[val].append(key.replace('/','\\'))

print(keyword_to_chap)

defaultdict(<class 'list'>, {'karamchand gandhi': ['1\\chapter1'], 'alias kaba gandhi': ['1\\chapter1'], 'uttamchand gandhi': ['1\\chapter1'], 'faithfulness': ['1\\chapter4'], 'smoking stumps': ['1\\chapter7'], 'cigarettes': ['1\\chapter7'], 'cigarette': ['1\\chapter7'], 'vaishnava faith': ['1\\chapter9'], 'ahmedabad': ['1\\chapter10', '5\\chapter9', '5\\chapter20'], 'matriculation examination': ['1\\chapter10'], 'kathiawad students': ['1\\chapter10'], 'ringworm': ['1\\chapter13'], 'vegetarianism': ['1\\chapter14', '1\\chapter16', '4\\chapter6'], 'dietary': ['1\\chapter14'], 'diet': ['1\\chapter14'], 'vegetarian': ['1\\chapter16'], 'narayan hemchandra': ['1\\chapter21'], 'time narayan hemchandra': ['1\\chapter21'], 'narayan': ['1\\chapter21'], 'eiffel tower': ['1\\chapter22'], 'legal maxims': ['1\\chapter24'], 'briefless barristers': ['2\\chapter3'], 'barrister': ['2\\chapter3', '2\\chapter4', '2\\chapter14', '4\\chapter46'], 'barristers': ['2\\chapter3'], 'bombay commissions': ['2\\ch

In [6]:
final_keywords = set()
all_keywords = list(mp.values())
for i in all_keywords:
    for j,val in i:
        if val>=0.45:
            final_keywords.add(j)
print(final_keywords)   
print(len(final_keywords))

{'bombay gokhale', 'maulana mazharul haq', 'ashram', 'kalelkar', 'kheda satyagraha', 'indigo', 'kheda satyagraha struggle', 'ashram children', 'household', 'laxatives', 'narayan hemchandra', 'word sadagraha', 'jalianwala bagh tragedy', 'loyalty', 'brahmacharya vow', 'result maganlal gandhi', 'punjab leaders', 'ashram walls', 'smoking stumps', 'bombay commissions', 'sheth haji muhammad haji dada', 'narayan', 'brahmacharya', 'vaishnava faith', 'gokhlae', 'zulus', 'punjab situation', 'natal legislative assembly', 'cigarettes', 'acharya ramadevji', 'cigarette', 'washing', 'indian evidence act', 'fasting', 'vow', 'treatment', 'maharshi devendranath', 'servant', 'professor kripalani', 'mohanlal pandya', 'quarantine officer', 'shankarlal parikh', 'abolition', 'swadeshi agitation', 'meanwhile delhi', 'plague', 'punjab government', 'gurukul', 'zulu', 'vegetarian', 'time narayan hemchandra', 'transvaal', 'court', 'laundry', 'hindu influence', 'insult', 'slivers', 'diet', 'quarantine', 'eiffel to

In [7]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def create_representative_doc(corpus,query):
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    paragraph = ""
    for i,score in enumerate(cos_scores):
        if score >0.40:
            paragraph += (corpus[i]+'.')

    return paragraph

In [8]:
keyword_to_chap

defaultdict(list,
            {'karamchand gandhi': ['1\\chapter1'],
             'alias kaba gandhi': ['1\\chapter1'],
             'uttamchand gandhi': ['1\\chapter1'],
             'faithfulness': ['1\\chapter4'],
             'smoking stumps': ['1\\chapter7'],
             'cigarettes': ['1\\chapter7'],
             'cigarette': ['1\\chapter7'],
             'vaishnava faith': ['1\\chapter9'],
             'ahmedabad': ['1\\chapter10', '5\\chapter9', '5\\chapter20'],
             'matriculation examination': ['1\\chapter10'],
             'kathiawad students': ['1\\chapter10'],
             'ringworm': ['1\\chapter13'],
             'vegetarianism': ['1\\chapter14', '1\\chapter16', '4\\chapter6'],
             'dietary': ['1\\chapter14'],
             'diet': ['1\\chapter14'],
             'vegetarian': ['1\\chapter16'],
             'narayan hemchandra': ['1\\chapter21'],
             'time narayan hemchandra': ['1\\chapter21'],
             'narayan': ['1\\chapter21'],
          

In [9]:
def split_sentence_from_text(key):
    file_loc = r"C:\Users\hp\OneDrive\Documents\topic_coherence\SOMEWT\part%s.txt"%key
    text = ""
    with open(file_loc) as f:
        text = f.read()

    res = re.findall(r"[^.!?]+", text)
    return res

mp_key_doc = {}
for key,val in keyword_to_chap.items():
    mp_key_doc[key] = ""
    print(f"{key}")
    for k in keyword_to_chap[key]:
        corpus = split_sentence_from_text(k)
#         doc = create_representative_doc(corpus,key)
        mp_key_doc[key] += create_representative_doc(corpus,key)

print(mp_key_doc)

karamchand gandhi
alias kaba gandhi
uttamchand gandhi
faithfulness
smoking stumps
cigarettes
cigarette
vaishnava faith
ahmedabad
matriculation examination
kathiawad students
ringworm
vegetarianism
dietary
diet
vegetarian
narayan hemchandra
time narayan hemchandra
narayan
eiffel tower
legal maxims
briefless barristers
barrister
barristers
bombay commissions
port natal
christian indians
prayers
prayer
transvaal
litigation
natal legislative assembly
durban
sheth haji muhammad
sheth haji muhammad haji dada
financially sheth abdulla haji
bombay high court
court
natal indian congress
natal indians
natal zulus
taxation
servant
household
loyalty
sir pherozeshah
calcutta
raja sir pyarimohan mukarji
bengal club
hindu wife
hindu husband
quarantine
quarantine officer
plague
brahmacharya vow
brahmacharya
vow
washing outfit
laundry
washing
boer war
sir pherozeshah mehta
babu bhupendranath basu
india club
professor ray
gokhlae
christian indian
babu kalicharan banerji
brahmo samaj
adi brahmo samaj
mah

In [10]:
mp_key_doc

{'karamchand gandhi': 'The Gandhis belong to the Bania caste and seem to have been originally grocers. Uttamchand Gandhi, alias Ota Gandhi, my grandfather, must have been a man of principle. Ota Gandhi married a second time, having lost his first wife. I do not think that in my childhood I ever felt or knew that these sons of Ota Gandhi were not all of the same mother. The fifth of these six brothers was Karamchand Gandhi, alias Kaba Gandhi, and the sixth was Tulsidas Gandhi. Kaba Gandhi was my father. He was for some time Prime Minister in Rajkot and then in Vankaner. Kaba Gandhi married four times in succession, having lost his wife each time by death. The Agent was angry and asked Kaba Gandhi to apologize. But when the Agent saw that Kaba Gandhi was adamant, he ordered him to be released.',
 'alias kaba gandhi': 'The Gandhis belong to the Bania caste and seem to have been originally grocers. Uttamchand Gandhi, alias Ota Gandhi, my grandfather, must have been a man of principle. Ota 

In [11]:
url = 'https://en.wikipedia.org/wiki/Mahatma_Gandhi'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)
#print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')

desired_ids = []
for link in soup.find_all('span', attrs={'class':'mw-headline'}):
    if link.get('id') is not None: 
        desired_ids.append(link.get('id'))
print(desired_ids)

def FetchParagraphBetweenIds(id1,id2):
    hElem = soup.find("span", {'id': id1})
    endElem = soup.find('span', {'id': id2})
    cntns = list(soup.find_all())

    my_lst = []
    inBetween = False
    for tag in cntns:
        if tag == hElem:
            inBetween = True
        if inBetween == True and tag.name == 'p':
            my_lst.append(tag.get_text())
        if tag == endElem:
            inBetween = False
            break
    return "".join(my_lst)

id_to_paragraph = {}
for i in range(len(desired_ids)-1):
    id_to_paragraph[desired_ids[i]] = FetchParagraphBetweenIds(desired_ids[i],desired_ids[i+1])

['Biography', 'Early_life_and_background', 'Three_years_in_London', 'Student_of_law', 'Vegetarianism_and_committee_work', 'Called_to_the_bar', 'Civil_rights_activist_in_South_Africa_(1893–1914)', 'Europeans,_Indians_and_Africans', 'Struggle_for_Indian_independence_(1915–1947)', 'Role_in_World_War_I', 'Champaran_agitations', 'Kheda_agitations', 'Khilafat_movement', 'Non-co-operation', 'Salt_Satyagraha_(Salt_March)', 'Gandhi_as_folk_hero', 'Negotiations', 'Round_Table_Conferences', 'Congress_politics', 'World_War_II_and_Quit_India_movement', 'Partition_and_independence', 'Death', 'Funeral_and_memorials', 'Principles,_practices,_and_beliefs', 'Influences', 'Leo_Tolstoy', 'Shrimad_Rajchandra', 'Religious_texts', 'Sufism', 'On_wars_and_nonviolence', 'Support_for_wars', 'Truth_and_Satyagraha', 'Nonviolence', 'On_inter-religious_relations', 'Buddhists,_Jains_and_Sikhs', 'Muslims', 'Christians', 'Jews', 'On_life,_society_and_other_application_of_his_ideas', 'Vegetarianism,_food,_and_animals', 

In [12]:
from sentence_transformers import SentenceTransformer, util
import torch

mp_sim_score = {}
embedder = SentenceTransformer('all-MiniLM-L6-v2')

for key1, val1 in id_to_paragraph.items():
    for key2,val2 in mp_key_doc.items():
        if val1 != '' or val2 != '':
            corpus_embeddings = embedder.encode(val1, convert_to_tensor=True)
            query_embedding = embedder.encode(val2, convert_to_tensor=True)
            cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0][0]
            mp_sim_score[str(key1)+'->'+str(key2)] = cos_scores
            print(f"{key1} {key2} {cos_scores}")

Biography karamchand gandhi -0.0009599626064300537
Biography alias kaba gandhi -0.0022823531180620193
Biography uttamchand gandhi -0.018190251663327217
Biography faithfulness 0.06764879077672958
Biography smoking stumps 0.023289170116186142
Biography cigarettes 0.014574429020285606
Biography cigarette 0.029046667739748955
Biography vaishnava faith 0.07004989683628082
Biography ahmedabad 0.03120323456823826
Biography matriculation examination 0.038604091852903366
Biography kathiawad students 0.010914658196270466
Biography ringworm 0.059086330235004425
Biography vegetarianism -0.022432539612054825
Biography dietary -0.026004010811448097
Biography diet 0.03618011623620987
Biography vegetarian -0.0024893414229154587
Biography narayan hemchandra 0.11311886459589005
Biography time narayan hemchandra 0.11311886459589005
Biography narayan 0.11311886459589005
Biography eiffel tower -0.003068678081035614
Biography legal maxims 0.002368747256696224
Biography briefless barristers 0.053083829581737

Early_life_and_background bombay high court 0.26862096786499023
Early_life_and_background court 0.31721511483192444
Early_life_and_background natal indian congress 0.32212942838668823
Early_life_and_background natal indians 0.32393878698349
Early_life_and_background natal zulus 0.3031131625175476
Early_life_and_background taxation 0.3712003827095032
Early_life_and_background servant 0.236469566822052
Early_life_and_background household 0.27649351954460144
Early_life_and_background loyalty 0.18290337920188904
Early_life_and_background sir pherozeshah 0.24992260336875916
Early_life_and_background calcutta 0.4871903359889984
Early_life_and_background raja sir pyarimohan mukarji 0.528088390827179
Early_life_and_background bengal club 0.4694560766220093
Early_life_and_background hindu wife 0.3771122097969055
Early_life_and_background hindu husband 0.3667162358760834
Early_life_and_background quarantine 0.16775591671466827
Early_life_and_background quarantine officer 0.16775591671466827
Earl

Three_years_in_London natal indian congress 0.04652971774339676
Three_years_in_London natal indians 0.06856448203325272
Three_years_in_London natal zulus 0.03498145937919617
Three_years_in_London taxation 0.0492592453956604
Three_years_in_London servant -0.0239095576107502
Three_years_in_London household 0.015707019716501236
Three_years_in_London loyalty 0.08914631605148315
Three_years_in_London sir pherozeshah 0.06951428204774857
Three_years_in_London calcutta 0.0916077271103859
Three_years_in_London raja sir pyarimohan mukarji 0.06713749468326569
Three_years_in_London bengal club 0.11318469047546387
Three_years_in_London hindu wife -0.028102228417992592
Three_years_in_London hindu husband -0.02267567440867424
Three_years_in_London quarantine 0.02981892228126526
Three_years_in_London quarantine officer 0.02981892228126526
Three_years_in_London plague 0.034504253417253494
Three_years_in_London brahmacharya vow -0.03929408639669418
Three_years_in_London brahmacharya 0.04897291213274002


Student_of_law brahmacharya vow 0.3350347578525543
Student_of_law brahmacharya 0.3882938623428345
Student_of_law vow 0.38015586137771606
Student_of_law washing outfit 0.12561973929405212
Student_of_law laundry 0.10803642868995667
Student_of_law washing 0.11634135246276855
Student_of_law boer war 0.2764458656311035
Student_of_law sir pherozeshah mehta 0.5553138852119446
Student_of_law babu bhupendranath basu 0.29742586612701416
Student_of_law india club 0.504741370677948
Student_of_law professor ray 0.0993245393037796
Student_of_law gokhlae 0.26263779401779175
Student_of_law christian indian 0.4096373915672302
Student_of_law babu kalicharan banerji 0.362032949924469
Student_of_law brahmo samaj 0.45596182346343994
Student_of_law adi brahmo samaj 0.45596182346343994
Student_of_law maharshi devendranath 0.5110809206962585
Student_of_law indian evidence act 0.41893500089645386
Student_of_law insult 0.18746283650398254
Student_of_law hindu influence 0.3093814551830292
Student_of_law hinduism

Vegetarianism_and_committee_work india club 0.41267159581184387
Vegetarianism_and_committee_work professor ray 0.061283327639102936
Vegetarianism_and_committee_work gokhlae 0.27616357803344727
Vegetarianism_and_committee_work christian indian 0.34171444177627563
Vegetarianism_and_committee_work babu kalicharan banerji 0.3064225912094116
Vegetarianism_and_committee_work brahmo samaj 0.38964319229125977
Vegetarianism_and_committee_work adi brahmo samaj 0.38964319229125977
Vegetarianism_and_committee_work maharshi devendranath 0.4093935489654541
Vegetarianism_and_committee_work indian evidence act 0.323256254196167
Vegetarianism_and_committee_work insult 0.18141481280326843
Vegetarianism_and_committee_work hindu influence 0.3355942368507385
Vegetarianism_and_committee_work hinduism 0.2929515838623047
Vegetarianism_and_committee_work hindu 0.3355942368507385
Vegetarianism_and_committee_work laxative medicines 0.14097456634044647
Vegetarianism_and_committee_work laxatives 0.1409745663404464

...

Explanatory_notes taxation 0.0492592453956604
Explanatory_notes servant -0.0239095576107502
Explanatory_notes household 0.015707019716501236
Explanatory_notes loyalty 0.08914631605148315
Explanatory_notes sir pherozeshah 0.06951428204774857
Explanatory_notes calcutta 0.0916077271103859
Explanatory_notes raja sir pyarimohan mukarji 0.06713749468326569
Explanatory_notes bengal club 0.11318469047546387
Explanatory_notes hindu wife -0.028102228417992592
Explanatory_notes hindu husband -0.02267567440867424
Explanatory_notes quarantine 0.02981892228126526
Explanatory_notes quarantine officer 0.02981892228126526
Explanatory_notes plague 0.034504253417253494
Explanatory_notes brahmacharya vow -0.03929408639669418
Explanatory_notes brahmacharya 0.04897291213274002
Explanatory_notes vow -0.010112682357430458
Explanatory_notes washing outfit 0.028545983135700226
Explanatory_notes laundry 0.005983835086226463
Explanatory_notes washing 0.02259335294365883
Explanatory_notes boer war 0.08632482588291

Citations hindu influence 0.03135158121585846
Citations hinduism 0.01598513498902321
Citations hindu 0.03135158121585846
Citations laxative medicines -0.012630259618163109
Citations laxatives -0.012630259618163109
Citations indian opinion 0.03900261968374252
Citations hindus 0.03509984165430069
Citations zulus 0.09236226975917816
Citations zulu 0.09236226975917816
Citations satyagraha 0.08024274557828903
Citations word sadagraha 0.09797369688749313
Citations result maganlal gandhi 0.06823975592851639
Citations curry powder -0.03435448929667473
Citations fasting 0.020692676305770874
Citations tolstoy farm 0.03510734438896179
Citations commanding officer 0.07591941952705383
Citations commanding 0.06795600801706314
Citations parsi rustomji 0.0603935644030571
Citations bombay gokhale 0.1442253291606903
Citations kakasaheb 0.05191207304596901
Citations kalelkar 0.040558110922575
Citations teachers family names 0.008551348000764847
Citations acharya ramadevji 0.04232988879084587
Citations gu

General_and_cited_references fasting 0.020692676305770874
General_and_cited_references tolstoy farm 0.03510734438896179
General_and_cited_references commanding officer 0.07591941952705383
General_and_cited_references commanding 0.06795600801706314
General_and_cited_references parsi rustomji 0.0603935644030571
General_and_cited_references bombay gokhale 0.1442253291606903
General_and_cited_references kakasaheb 0.05191207304596901
General_and_cited_references kalelkar 0.040558110922575
General_and_cited_references teachers family names 0.008551348000764847
General_and_cited_references acharya ramadevji 0.04232988879084587
General_and_cited_references gurukul 0.09080198407173157
General_and_cited_references gujarat 0.10045962780714035
General_and_cited_references satyagraha ashram -0.009425099939107895
General_and_cited_references amritlal thakkar 0.11795857548713684
General_and_cited_references ashram 0.04464017227292061
General_and_cited_references abolition 0.09956222027540207
General_

Books meanwhile delhi 0.017587579786777496
Books kheda satyagraha struggle 0.05496685951948166
Books jalianwala bagh tragedy 0.060470372438430786
Books punjab situation 0.048505283892154694
Books punjab government 0.06041702628135681
Books jalianwala bagh massacre 0.03590439260005951
Books punjab leaders 0.024272024631500244
Books pandit motilal nehru 0.04058244824409485
Books pandit 0.07635363191366196
Books slivers 0.03676450252532959
Books swadeshi agitation 0.083299919962883
Books swadeshi movement 0.083299919962883
Books khilafat agitation 0.05423731356859207
Scholarly_articles karamchand gandhi -0.0009599626064300537
Scholarly_articles alias kaba gandhi -0.0022823531180620193
Scholarly_articles uttamchand gandhi -0.018190251663327217
Scholarly_articles faithfulness 0.06764879077672958
Scholarly_articles smoking stumps 0.023289170116186142
Scholarly_articles cigarettes 0.014574429020285606
Scholarly_articles cigarette 0.029046667739748955
Scholarly_articles vaishnava faith 0.07004

Primary_sources cigarettes 0.014574429020285606
Primary_sources cigarette 0.029046667739748955
Primary_sources vaishnava faith 0.07004989683628082
Primary_sources ahmedabad 0.03120323456823826
Primary_sources matriculation examination 0.038604091852903366
Primary_sources kathiawad students 0.010914658196270466
Primary_sources ringworm 0.059086330235004425
Primary_sources vegetarianism -0.022432539612054825
Primary_sources dietary -0.026004010811448097
Primary_sources diet 0.03618011623620987
Primary_sources vegetarian -0.0024893414229154587
Primary_sources narayan hemchandra 0.11311886459589005
Primary_sources time narayan hemchandra 0.11311886459589005
Primary_sources narayan 0.11311886459589005
Primary_sources eiffel tower -0.003068678081035614
Primary_sources legal maxims 0.002368747256696224
Primary_sources briefless barristers 0.05308382958173752
Primary_sources barrister 0.01604798436164856
Primary_sources barristers 0.04784006625413895
Primary_sources bombay commissions -0.00690

In [13]:
with open("mp_key_doc2.json", "w") as outfile:
    json.dump(mp_key_doc, outfile)

In [14]:
max(mp_sim_score.values())

tensor(0.6921)

In [15]:
mp_sim_score_range = {}
for key,val in mp_sim_score.items():
    if val >= 0.2 and val <= 0.70:
        mp_sim_score_range[key] = val

In [16]:
mp_sim_score_range
maxval = max(mp_sim_score.values())
for k,v in mp_sim_score_range.items():
    mp_sim_score_range[k] = v/maxval

In [17]:
mp_sim_score_range

{'Early_life_and_background->karamchand gandhi': tensor(1.),
 'Early_life_and_background->alias kaba gandhi': tensor(0.9913),
 'Early_life_and_background->uttamchand gandhi': tensor(0.9835),
 'Early_life_and_background->ahmedabad': tensor(0.6406),
 'Early_life_and_background->kathiawad students': tensor(0.3685),
 'Early_life_and_background->narayan hemchandra': tensor(0.4487),
 'Early_life_and_background->time narayan hemchandra': tensor(0.4487),
 'Early_life_and_background->narayan': tensor(0.4487),
 'Early_life_and_background->briefless barristers': tensor(0.4100),
 'Early_life_and_background->barrister': tensor(0.4200),
 'Early_life_and_background->barristers': tensor(0.4505),
 'Early_life_and_background->bombay commissions': tensor(0.3557),
 'Early_life_and_background->christian indians': tensor(0.4713),
 'Early_life_and_background->transvaal': tensor(0.4250),
 'Early_life_and_background->natal legislative assembly': tensor(0.3205),
 'Early_life_and_background->durban': tensor(0.33

In [18]:
mp_sim_score_high = {}
mp_sim_score_low = {}

for k,v in mp_sim_score_range.items():
    if v>=0.5:
        mp_sim_score_high[k] = v
    elif v<0.5:
        mp_sim_score_low[k] = v
        

In [19]:
mp_sim_score_high = {k: float(v) for k, v in mp_sim_score_high.items()}
with open("mp_sim_score_high.json", "w") as outfile:
    json.dump(mp_sim_score_high, outfile)

In [20]:
mp_sim_score_high

{'Early_life_and_background->karamchand gandhi': 1.0,
 'Early_life_and_background->alias kaba gandhi': 0.9913423657417297,
 'Early_life_and_background->uttamchand gandhi': 0.9834907650947571,
 'Early_life_and_background->ahmedabad': 0.6406016945838928,
 'Early_life_and_background->sheth haji muhammad': 0.6207416653633118,
 'Early_life_and_background->sheth haji muhammad haji dada': 0.6250967979431152,
 'Early_life_and_background->financially sheth abdulla haji': 0.5857303142547607,
 'Early_life_and_background->taxation': 0.5363202691078186,
 'Early_life_and_background->calcutta': 0.7039056420326233,
 'Early_life_and_background->raja sir pyarimohan mukarji': 0.762996256351471,
 'Early_life_and_background->bengal club': 0.6782827377319336,
 'Early_life_and_background->hindu wife': 0.5448618531227112,
 'Early_life_and_background->hindu husband': 0.529841423034668,
 'Early_life_and_background->sir pherozeshah mehta': 0.7401658296585083,
 'Early_life_and_background->babu bhupendranath basu'

In [21]:
mp_sim_score_low = {k: float(v) for k, v in mp_sim_score_low.items()}
with open("mp_sim_score_low.json", "w") as outfile:
    json.dump(mp_sim_score_low, outfile)

In [22]:
mp_sim_score_low

{'Early_life_and_background->kathiawad students': 0.36845269799232483,
 'Early_life_and_background->narayan hemchandra': 0.4486844539642334,
 'Early_life_and_background->time narayan hemchandra': 0.4486844539642334,
 'Early_life_and_background->narayan': 0.4486844539642334,
 'Early_life_and_background->briefless barristers': 0.40996038913726807,
 'Early_life_and_background->barrister': 0.4199908375740051,
 'Early_life_and_background->barristers': 0.4504583179950714,
 'Early_life_and_background->bombay commissions': 0.3556618094444275,
 'Early_life_and_background->christian indians': 0.4712912440299988,
 'Early_life_and_background->transvaal': 0.42504969239234924,
 'Early_life_and_background->natal legislative assembly': 0.3205198645591736,
 'Early_life_and_background->durban': 0.3379760980606079,
 'Early_life_and_background->bombay high court': 0.3881107568740845,
 'Early_life_and_background->court': 0.45832088589668274,
 'Early_life_and_background->natal indian congress': 0.4654212296

In [12]:
with open('mp_sim_score_low.json') as json_file:
    mp_sim_score_low = json.load(json_file)
    
with open("mp_sim_score_high.json") as outfile:
    mp_sim_score_high = json.load(outfile)
    
with open("mp_key_doc2.json") as outfile:
    mp_key_doc = json.load(outfile)

In [8]:
mp_sim_score_high_80 = {}
for k,v in mp_sim_score_high.items():
#     print(v)
    if v>0.80:
        mp_sim_score_high_80[k] = v
print(mp_sim_score_high_80)

{'Early_life_and_background->karamchand gandhi': 1.0, 'Early_life_and_background->alias kaba gandhi': 0.9913423657417297, 'Early_life_and_background->uttamchand gandhi': 0.9834907650947571, 'Early_life_and_background->maharshi devendranath': 0.89777672290802, 'Early_life_and_background->rajendrababu': 0.8018625378608704, 'Early_life_and_background->gangadharrao deshpande': 0.8573912382125854, 'Student_of_law->karamchand gandhi': 0.893534243106842, 'Student_of_law->alias kaba gandhi': 0.9235007166862488, 'Student_of_law->uttamchand gandhi': 0.9357511401176453, 'Student_of_law->sir pherozeshah mehta': 0.8023324012756348, 'Called_to_the_bar->uttamchand gandhi': 0.8357126116752625, 'Called_to_the_bar->barrister': 0.8322190046310425, 'Called_to_the_bar->barristers': 0.8188490271568298, 'Called_to_the_bar->maharshi devendranath': 0.8085132837295532, 'Civil_rights_activist_in_South_Africa_(1893–1914)->alias kaba gandhi': 0.8052709698677063, 'Civil_rights_activist_in_South_Africa_(1893–1914)->

## Sectionwise hyperlinks

In [14]:
def FetchURLSBetweenIds(id1,id2):
    hElem = soup.find("span", {'id': id1})
    endElem = soup.find('span', {'id': id2})
    cntns = list(soup.find_all())

    linkToScrape = []
    inBetween = False
    for tag in cntns:
        if tag == hElem:
            inBetween = True
        if inBetween == True and tag.name == 'p':
#             my_lst.append(tag.get_text())
            allLinks = tag.find_all('a')
            
            for link in allLinks:
                if link.get('href') is not None and link.get('href').find("/wiki/")== 0: 
                    linkToScrape.append(link['href'].replace('/wiki/',""))
        if tag == endElem:
            inBetween = False
            break
    return set(linkToScrape)

FetchURLSBetweenIds("Early_life_and_background","Three_years_in_London")

{'Ahmedabad',
 'Alfred_High_School_(Rajkot)',
 'Arranged_marriage',
 'Bania_(caste)',
 'Bhagavad_Gita',
 'Bhagavata_Purana',
 'Bhavnagar_State',
 'Bible',
 'British_Raj',
 'Devdas_Gandhi',
 'Dewan',
 'Gujarati_people',
 'Harilal_Gandhi',
 'Harishchandra',
 'Hindu',
 'Junagadh_State',
 'Karamchand_Uttamchand_Gandhi',
 'Kasturba_Gandhi',
 'Kathiawar',
 'Kathiawar_Agency',
 'Krishna',
 'Kutiyana',
 'Manilal_Gandhi',
 'Modh',
 'Porbandar',
 'Porbandar_State',
 'Pranami',
 'Princely_state',
 'Quran',
 'Rajkot',
 'Ramdas_Gandhi',
 'Samaldas_College',
 'Shravan#In_Hindu_epics',
 'Vaishnava',
 'Vaishnavism',
 'Vaishya',
 'Varna_(Hinduism)',
 'Vedas'}

In [15]:
mp_urls_section_mp  = {}
for i in range(len(desired_ids)-1):
    mp_urls_section_mp[desired_ids[i]] = FetchURLSBetweenIds(desired_ids[i],desired_ids[i+1])
print(mp_urls_section_mp)

{'Biography': set(), 'Early_life_and_background': {'Porbandar', 'Bania_(caste)', 'Quran', 'Devdas_Gandhi', 'Karamchand_Uttamchand_Gandhi', 'Vaishnavism', 'Vaishnava', 'Manilal_Gandhi', 'Dewan', 'Kutiyana', 'Pranami', 'Vaishya', 'Samaldas_College', 'Gujarati_people', 'Ramdas_Gandhi', 'Alfred_High_School_(Rajkot)', 'Princely_state', 'Kathiawar_Agency', 'Arranged_marriage', 'Ahmedabad', 'Shravan#In_Hindu_epics', 'Harilal_Gandhi', 'Bhagavad_Gita', 'Harishchandra', 'Hindu', 'British_Raj', 'Modh', 'Bhavnagar_State', 'Bhagavata_Purana', 'Bible', 'Junagadh_State', 'Varna_(Hinduism)', 'Kasturba_Gandhi', 'Vedas', 'Rajkot', 'Kathiawar', 'Porbandar_State', 'Krishna'}, 'Three_years_in_London': set(), 'Student_of_law': {'Henry_Morley', 'Brahmin', 'University_College,_London', 'Henry_Edward_Manning', 'London_dock_strike,_1889', 'Barrister', 'Inner_Temple'}, 'Vegetarianism_and_committee_work': {'Buddhist', 'Captain_of_industry', 'West_Ham_United', 'Thomas_Allinson', 'Bhagavad_Gita', 'Theosophical_Soci

In [16]:
section_wise_keyword = collections.defaultdict(set)
for k,v in mp_sim_score_high.items():
    section_name, keyword = k.split('->')[0],k.split('->')[1]
    section_wise_keyword[section_name].add(keyword.title().replace(" ", "_"))

 ### Now we have two maps  section_wise_keyword and mp_urls_section_mp

In [17]:
section_wise_keyword

defaultdict(set,
            {'Early_life_and_background': {'Adi_Brahmo_Samaj',
              'Ahmedabad',
              'Alias_Kaba_Gandhi',
              'Amritlal_Thakkar',
              'Babu_Bhupendranath_Basu',
              'Babu_Kalicharan_Banerji',
              'Bengal_Club',
              'Bombay_Gokhale',
              'Brahmo_Samaj',
              'Calcutta',
              'Financially_Sheth_Abdulla_Haji',
              'Gangadharrao_Deshpande',
              'Gujarat',
              'Hindu_Husband',
              'Hindu_Wife',
              'India_Club',
              'Indian_Evidence_Act',
              'Indian_Opinion',
              'Jalianwala_Bagh_Massacre',
              'Janakdharibabu',
              'Kakasaheb',
              'Kalelkar',
              'Karamchand_Gandhi',
              'Kheda_Satyagraha',
              'Khilafat_Agitation',
              'Maharshi_Devendranath',
              'Maulana_Mazharul_Haq',
              'Mohanlal_Pandya',
              

In [17]:
reamining_keywords = {}
for k ,v in section_wise_keyword.items():
    reamining_keywords[k] = v - mp_urls_section_mp[k]
    print(f"{k}  {len(section_wise_keyword[k])}  {len(reamining_keywords[k])}")

Early_life_and_background  6  6
Student_of_law  4  4
Called_to_the_bar  4  4
Civil_rights_activist_in_South_Africa_(1893–1914)  2  2
Champaran_agitations  1  1
Kheda_agitations  1  1
Khilafat_movement  1  1
Non-co-operation  1  1
Gandhi_as_folk_hero  1  1
Partition_and_independence  1  1
Death  3  3
Shrimad_Rajchandra  1  1
Religious_texts  1  1
Support_for_wars  2  2
Truth_and_Satyagraha  3  2
Christians  1  1
Vegetarianism,_food,_and_animals  2  2
Women  6  6
Brahmacharya:_abstinence_from_sex_and_food  1  1
Untouchability_and_castes  1  1
Literary_works  1  0
Descendants  2  2


In [19]:
for k ,v in section_wise_keyword.items():
    print(f"{k}  {len(v)}")

Early_life_and_background  49
Student_of_law  60
Vegetarianism_and_committee_work  40
Called_to_the_bar  55
Civil_rights_activist_in_South_Africa_(1893–1914)  45
Europeans,_Indians_and_Africans  36
Struggle_for_Indian_independence_(1915–1947)  25
Role_in_World_War_I  13
Champaran_agitations  41
Kheda_agitations  35
Khilafat_movement  40
Non-co-operation  50
Salt_Satyagraha_(Salt_March)  30
Gandhi_as_folk_hero  52
Negotiations  26
Round_Table_Conferences  42
Congress_politics  22
World_War_II_and_Quit_India_movement  26
Partition_and_independence  38
Death  34
Funeral_and_memorials  22
Principles,_practices,_and_beliefs  25
Influences  46
Leo_Tolstoy  24
Shrimad_Rajchandra  48
Religious_texts  25
Sufism  34
Support_for_wars  19
Truth_and_Satyagraha  28
Nonviolence  33
Buddhists,_Jains_and_Sikhs  24
Muslims  38
Christians  15
Jews  20
Vegetarianism,_food,_and_animals  24
Fasting  17
Women  44
Brahmacharya:_abstinence_from_sex_and_food  29
Untouchability_and_castes  28
Nai_Talim,_basic_ed

In [20]:
def fetch_dates_from_content(content):
    regex= "\d{4}"
    dates = re.findall(regex, content)
    year_list = []
    try:
        for date in dates:
            if int(date) < 1960 and int(date) > 1800:                    
                #print date
                year_list.append(date)
    except:
        pass
    return [min(year_list),max(year_list)] if year_list else []
wiki_section_date_range = {}
for k,v in id_to_paragraph.items():
    print(fetch_dates_from_content(v))
    wiki_section_date_range[k] = fetch_dates_from_content(v)

[]
['1822', '1914']
[]
['1888', '1889']
['1875', '1927']
['1891', '1893']
['1893', '1915']
['1895', '1910']
['1915', '1947']
['1906', '1918']
['1917', '1917']
['1918', '1918']
['1917', '1922']
['1909', '1924']
['1827', '1930']
['1890', '1920']
['1930', '1931']
['1914', '1948']
['1934', '1938']
['1942', '1947']
['1946', '1947']
['1948', '1949']
['1942', '1948']
[]
['1849', '1935']
['1908', '1910']
['1891', '1930']
['1930', '1930']
[]
[]
['1899', '1941']
['1920', '1945']
['1940', '1946']
[]
[]
['1919', '1940']
[]
['1917', '1947']
[]
['1890', '1890']
['1932', '1948']
[]
['1906', '1947']
['1920', '1956']
['1937', '1947']
[]
[]
['1930', '1930']
['1936', '1936']
['1909', '1934']
[]
['1924', '1948']
['1948', '1948']
['1930', '1948']
['1944', '1947']
['1869', '1948']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [21]:
wiki_section_date_range

{'Biography': [],
 'Early_life_and_background': ['1822', '1914'],
 'Three_years_in_London': [],
 'Student_of_law': ['1888', '1889'],
 'Vegetarianism_and_committee_work': ['1875', '1927'],
 'Called_to_the_bar': ['1891', '1893'],
 'Civil_rights_activist_in_South_Africa_(1893–1914)': ['1893', '1915'],
 'Europeans,_Indians_and_Africans': ['1895', '1910'],
 'Struggle_for_Indian_independence_(1915–1947)': ['1915', '1947'],
 'Role_in_World_War_I': ['1906', '1918'],
 'Champaran_agitations': ['1917', '1917'],
 'Kheda_agitations': ['1918', '1918'],
 'Khilafat_movement': ['1917', '1922'],
 'Non-co-operation': ['1909', '1924'],
 'Salt_Satyagraha_(Salt_March)': ['1827', '1930'],
 'Gandhi_as_folk_hero': ['1890', '1920'],
 'Negotiations': ['1930', '1931'],
 'Round_Table_Conferences': ['1914', '1948'],
 'Congress_politics': ['1934', '1938'],
 'World_War_II_and_Quit_India_movement': ['1942', '1947'],
 'Partition_and_independence': ['1946', '1947'],
 'Death': ['1948', '1949'],
 'Funeral_and_memorials': 

In [4]:
book ="SOMEWT"
def get_year_to_content(book):
    dir_path = './%s/part'% book
    date_dict = {}
    year_mode_dict = {}
    # for part in range(1,Parts+1):
    rootdir = r"C:\Users\hp\OneDrive\Documents\topic_coherence\%s"% book
    part = 1
    for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
        x = 1
        #print("part:" + str(part))
        date_dict[str(part)] = {}
        year_mode_dict[str(part)] = {}
        #print(dir_path + str(part) + '/')
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
    #     print('no.of_chapters:' + str(no_of_chapters))
        while x <= no_of_chapters:
            target_x = no_of_chapters+1
            for i in range(x,target_x):
                #print("chapter:" + str(i))
                with open(dir_path + str(part) + '/chapter'+ str(i) + '.txt', 'r', encoding='utf-8') as content_file:
                    content = content_file.read()

                regex= "\d{4}"
                dates = re.findall(regex, content)
                mdates = []
                year_list = []
                try:
                    for date in dates:
                        if int(date) < 1960 and int(date) > 1800:                    
                            #print date
                            year_list.append(date)
                            mdates.append(date)                       
                except:
                    pass
                date_dict[str(part)][str(i)] = mdates
                try:
                    year_mode_dict[str(part)][str(i)] = max(set(year_list), key=year_list.count)
                except:
                    year_mode_dict[str(part)][str(i)] = -1
            x = target_x
            part += 1

    # assigning chapter the date with closest chapter
    def closest(lst, K):
        return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]


    #for part in range(1,Parts+1):
    part = 1
    for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
        x = 1
        # checking which chapters has date assigned
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
        chapters_with_date = []
        while x <= no_of_chapters:
            target_x = no_of_chapters+1
            for i in range(x,target_x):
                # print("chapter:" + str(i))
                if year_mode_dict[str(part)][str(i)] != -1:
                    chapters_with_date.append(i)
            x = target_x
        x = 1
        # assigning date to chapter which is closest to it
        while x <= no_of_chapters:
            target_x = no_of_chapters+1
            for i in range(x,target_x):
                #print("chapter:" + str(i))
                k = closest(chapters_with_date,i)
                #print("K:",k)
                year_mode_dict[str(part)][str(i)] = year_mode_dict[str(part)][str(k)]
            x = target_x
        part += 1

    year_to_chap = collections.defaultdict(list)  # year -> list of folder paths
    for key in year_mode_dict.keys():
        folder = f"{key}"
        for k2 in year_mode_dict[key].keys():
            file = folder
            file += f"\\chapter{k2}"
            val = year_mode_dict[key][k2]
            year_to_chap[val].append(file)

    return year_to_chap

In [5]:
year_to_chap = get_year_to_content(book)

In [3]:
keyword_to_chap = {'karamchand gandhi': ['1\\chapter1'], 'alias kaba gandhi': ['1\\chapter1'], 'uttamchand gandhi': ['1\\chapter1'], 'faithfulness': ['1\\chapter4'], 'smoking stumps': ['1\\chapter7'], 'cigarettes': ['1\\chapter7'], 'cigarette': ['1\\chapter7'], 'vaishnava faith': ['1\\chapter9'], 'ahmedabad': ['1\\chapter10', '5\\chapter9', '5\\chapter20'], 'matriculation examination': ['1\\chapter10'], 'kathiawad students': ['1\\chapter10'], 'ringworm': ['1\\chapter13'], 'vegetarianism': ['1\\chapter14', '1\\chapter16', '4\\chapter6'], 'dietary': ['1\\chapter14'], 'diet': ['1\\chapter14'], 'vegetarian': ['1\\chapter16'], 'narayan hemchandra': ['1\\chapter21'], 'time narayan hemchandra': ['1\\chapter21'], 'narayan': ['1\\chapter21'], 'eiffel tower': ['1\\chapter22'], 'legal maxims': ['1\\chapter24'], 'briefless barristers': ['2\\chapter3'], 'barrister': ['2\\chapter3', '2\\chapter4', '2\\chapter14', '4\\chapter46'], 'barristers': ['2\\chapter3'], 'bombay commissions': ['2\\chapter4'], 'port natal': ['2\\chapter7'], 'christian indians': ['2\\chapter8'], 'prayers': ['2\\chapter11'], 'prayer': ['2\\chapter11', '2\\chapter15'], 'transvaal': ['2\\chapter13', '3\\chapter4', '4\\chapter2', '4\\chapter46'], 'litigation': ['2\\chapter14'], 'natal legislative assembly': ['2\\chapter16'], 'durban': ['2\\chapter16'], 'sheth haji muhammad': ['2\\chapter17'], 'sheth haji muhammad haji dada': ['2\\chapter17'], 'financially sheth abdulla haji': ['2\\chapter17'], 'bombay high court': ['2\\chapter18'], 'court': ['2\\chapter18'], 'natal indian congress': ['2\\chapter19', '2\\chapter20'], 'natal indians': ['2\\chapter21'], 'natal zulus': ['2\\chapter21'], 'taxation': ['2\\chapter21'], 'servant': ['2\\chapter23'], 'household': ['2\\chapter23'], 'loyalty': ['2\\chapter26'], 'sir pherozeshah': ['2\\chapter28'], 'calcutta': ['2\\chapter29'], 'raja sir pyarimohan mukarji': ['2\\chapter29'], 'bengal club': ['2\\chapter29'], 'hindu wife': ['3\\chapter1'], 'hindu husband': ['3\\chapter1'], 'quarantine': ['3\\chapter2'], 'quarantine officer': ['3\\chapter2'], 'plague': ['3\\chapter2'], 'brahmacharya vow': ['3\\chapter7', '3\\chapter8'], 'brahmacharya': ['3\\chapter7', '4\\chapter30'], 'vow': ['3\\chapter8'], 'washing outfit': ['3\\chapter9'], 'laundry': ['3\\chapter9'], 'washing': ['3\\chapter9'], 'boer war': ['3\\chapter10', '4\\chapter39'], 'sir pherozeshah mehta': ['3\\chapter13'], 'babu bhupendranath basu': ['3\\chapter14'], 'india club': ['3\\chapter16'], 'professor ray': ['3\\chapter17'], 'gokhlae': ['3\\chapter18'], 'christian indian': ['3\\chapter18'], 'babu kalicharan banerji': ['3\\chapter18'], 'brahmo samaj': ['3\\chapter19'], 'adi brahmo samaj': ['3\\chapter19'], 'maharshi devendranath': ['3\\chapter19'], 'indian evidence act': ['3\\chapter21'], 'insult': ['4\\chapter3'], 'hindu influence': ['4\\chapter5'], 'hinduism': ['4\\chapter5'], 'hindu': ['4\\chapter5'], 'laxative medicines': ['4\\chapter7'], 'laxatives': ['4\\chapter7'], 'indian opinion': ['4\\chapter8', '4\\chapter19'], 'hindus': ['4\\chapter14'], 'zulus': ['4\\chapter25'], 'zulu': ['4\\chapter25'], 'satyagraha': ['4\\chapter26', '5\\chapter24'], 'word sadagraha': ['4\\chapter26'], 'result maganlal gandhi': ['4\\chapter26'], 'curry powder': ['4\\chapter29'], 'fasting': ['4\\chapter31'], 'tolstoy farm': ['4\\chapter35'], 'commanding officer': ['4\\chapter40'], 'commanding': ['4\\chapter40'], 'parsi rustomji': ['4\\chapter47'], 'bombay gokhale': ['5\\chapter2'], 'kakasaheb': ['5\\chapter4'], 'kalelkar': ['5\\chapter4'], 'teachers family names': ['5\\chapter4'], 'acharya ramadevji': ['5\\chapter8'], 'gurukul': ['5\\chapter8'], 'gujarat': ['5\\chapter9'], 'satyagraha ashram': ['5\\chapter9'], 'amritlal thakkar': ['5\\chapter10'], 'ashram': ['5\\chapter10', '5\\chapter21'], 'abolition': ['5\\chapter11'], 'indigo': ['5\\chapter12'], 'professor kripalani': ['5\\chapter13'], 'maulana mazharul haq': ['5\\chapter13'], 'champaran inquiry': ['5\\chapter16'], 'champaran satyagraha': ['5\\chapter16'], 'rajendrababu': ['5\\chapter17'], 'janakdharibabu': ['5\\chapter17'], 'gangadharrao deshpande': ['5\\chapter17'], 'sanitation work': ['5\\chapter18'], 'sanitation': ['5\\chapter18'], 'treatment': ['5\\chapter18'], 'bihar government': ['5\\chapter19'], 'ashram walls': ['5\\chapter21'], 'ashram children': ['5\\chapter21'], 'kheda satyagraha': ['5\\chapter23'], 'mohanlal pandya': ['5\\chapter23'], 'shankarlal parikh': ['5\\chapter23'], 'satyagrahi volunteers': ['5\\chapter24'], 'satyagrahi': ['5\\chapter24'], 'mamlatdar': ['5\\chapter25'], 'hartal': ['5\\chapter31'], 'swami shraddhanandji': ['5\\chapter31'], 'meanwhile delhi': ['5\\chapter31'], 'kheda satyagraha struggle': ['5\\chapter33'], 'jalianwala bagh tragedy': ['5\\chapter34'], 'punjab situation': ['5\\chapter34'], 'punjab government': ['5\\chapter35', '5\\chapter37'], 'jalianwala bagh massacre': ['5\\chapter35', '5\\chapter38'], 'punjab leaders': ['5\\chapter35'], 'pandit motilal nehru': ['5\\chapter37'], 'pandit': ['5\\chapter37'], 'slivers': ['5\\chapter40'], 'swadeshi agitation': ['5\\chapter41'], 'swadeshi movement': ['5\\chapter41'], 'khilafat agitation': ['5\\chapter42']}

In [18]:
# year_to_chap
chap_to_year = {}
for k,v in year_to_chap.items():
    for l in v:
        chap_to_year[l]= int(k)
# chap_to_year
keyword_to_year = collections.defaultdict(list)
for k,v in keyword_to_chap.items():
    for l in v:
        keyword_to_year[k.title().replace(" ", "_")].append(chap_to_year[l])
print(keyword_to_year)

defaultdict(<class 'list'>, {'Karamchand_Gandhi': [1869], 'Alias_Kaba_Gandhi': [1869], 'Uttamchand_Gandhi': [1869], 'Faithfulness': [1869], 'Smoking_Stumps': [1887], 'Cigarettes': [1887], 'Cigarette': [1887], 'Vaishnava_Faith': [1887], 'Ahmedabad': [1887, 1915, 1915], 'Matriculation_Examination': [1887], 'Kathiawad_Students': [1887], 'Ringworm': [1887], 'Vegetarianism': [1887, 1890, 1893], 'Dietary': [1887], 'Diet': [1887], 'Vegetarian': [1890], 'Narayan_Hemchandra': [1890], 'Time_Narayan_Hemchandra': [1890], 'Narayan': [1890], 'Eiffel_Tower': [1890], 'Legal_Maxims': [1857], 'Briefless_Barristers': [1893], 'Barrister': [1893, 1893, 1885, 1890], 'Barristers': [1893], 'Bombay_Commissions': [1893], 'Port_Natal': [1893], 'Christian_Indians': [1924], 'Prayers': [1893], 'Prayer': [1893, 1885], 'Transvaal': [1885, 1897, 1893, 1890], 'Litigation': [1885], 'Natal_Legislative_Assembly': [1893], 'Durban': [1893], 'Sheth_Haji_Muhammad': [1893], 'Sheth_Haji_Muhammad_Haji_Dada': [1893], 'Financially

In [27]:
# section_wise_keyword  -> 
# keyword_to_year ->
# wiki_section_date_range  ->

# section_keyword_temporal = {}
# for k,v in section_wise_keyword.items():
#     print(f"{k}  {v}")
#     for words in v:
#         for years in keyword_to_year[words]:
#             if years in range(int(wiki_section_date_range[k][0]),int(wiki_section_date_range[k][1])):
#                 print(f"{k}   -> {v}")